In [1]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets(".", one_hot=True, reshape=False)

import tensorflow as tf

# Parameters
learning_rate = 0.00001
epochs = 10
batch_size = 128

# Number of samples to calculate validation and accuracy
# Decrease this if you're running out of memory to calculate accuracy
test_valid_size = 256

# Network Parameters
n_classes = 10  # MNIST total classes (0-9 digits)
dropout = 0.75  # Dropout, probability to keep units

# Store layers weight & bias
weights = {
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    'out': tf.Variable(tf.random_normal([1024, n_classes]))}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))}

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting ./train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting ./train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ./t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting ./t10k-labels-idx1-ubyte.gz


In [2]:
def conv2d(x, W, b, strides=1):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

In [3]:
def maxpool2d(x, k=2):
    return tf.nn.max_pool(
        x,
        ksize=[1, k, k, 1],
        strides=[1, k, k, 1],
        padding='SAME')

In [4]:
def conv_net(x, weights, biases, dropout):
    # Layer 1 - 28*28*1 to 14*14*32
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    conv1 = maxpool2d(conv1, k=2)

    # Layer 2 - 14*14*32 to 7*7*64
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    conv2 = maxpool2d(conv2, k=2)

    # Fully connected layer - 7*7*64 to 1024
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output Layer - class prediction - 1024 to 10
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

In [ ]:
# tf Graph input
x = tf.placeholder(tf.float32, [None, 28, 28, 1])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32)

# Model
logits = conv_net(x, weights, biases, keep_prob)

# Define loss and optimizer
cost = tf.reduce_mean(\
    tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)\
    .minimize(cost)

# Accuracy
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf. global_variables_initializer()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    for epoch in range(epochs):
        for batch in range(mnist.train.num_examples//batch_size):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            sess.run(optimizer, feed_dict={
                x: batch_x,
                y: batch_y,
                keep_prob: dropout})

            # Calculate batch loss and accuracy
            loss = sess.run(cost, feed_dict={
                x: batch_x,
                y: batch_y,
                keep_prob: 1.})
            valid_acc = sess.run(accuracy, feed_dict={
                x: mnist.validation.images[:test_valid_size],
                y: mnist.validation.labels[:test_valid_size],
                keep_prob: 1.})

            print('Epoch {:>2}, Batch {:>3} -'
                  'Loss: {:>10.4f} Validation Accuracy: {:.6f}'.format(
                epoch + 1,
                batch + 1,
                loss,
                valid_acc))

    # Calculate Test Accuracy
    test_acc = sess.run(accuracy, feed_dict={
        x: mnist.test.images[:test_valid_size],
        y: mnist.test.labels[:test_valid_size],
        keep_prob: 1.})
    print('Testing Accuracy: {}'.format(test_acc))

Epoch  1, Batch   1 -Loss: 64357.8438 Validation Accuracy: 0.132812
Epoch  1, Batch   2 -Loss: 57765.6406 Validation Accuracy: 0.136719
Epoch  1, Batch   3 -Loss: 44333.1914 Validation Accuracy: 0.144531
Epoch  1, Batch   4 -Loss: 39131.3047 Validation Accuracy: 0.164062
Epoch  1, Batch   5 -Loss: 35829.4258 Validation Accuracy: 0.125000
Epoch  1, Batch   6 -Loss: 32560.0098 Validation Accuracy: 0.140625
Epoch  1, Batch   7 -Loss: 29611.0938 Validation Accuracy: 0.144531
Epoch  1, Batch   8 -Loss: 28942.0234 Validation Accuracy: 0.187500
Epoch  1, Batch   9 -Loss: 25708.6055 Validation Accuracy: 0.187500
Epoch  1, Batch  10 -Loss: 25839.9609 Validation Accuracy: 0.183594
Epoch  1, Batch  11 -Loss: 24538.1836 Validation Accuracy: 0.195312
Epoch  1, Batch  12 -Loss: 19993.8926 Validation Accuracy: 0.207031
Epoch  1, Batch  13 -Loss: 20675.7266 Validation Accuracy: 0.210938
Epoch  1, Batch  14 -Loss: 17845.0449 Validation Accuracy: 0.218750
Epoch  1, Batch  15 -Loss: 19462.8867 Validation

Epoch  1, Batch 122 -Loss:  4394.0913 Validation Accuracy: 0.667969
Epoch  1, Batch 123 -Loss:  5191.1025 Validation Accuracy: 0.664062
Epoch  1, Batch 124 -Loss:  3739.1826 Validation Accuracy: 0.660156
Epoch  1, Batch 125 -Loss:  4428.4893 Validation Accuracy: 0.671875
Epoch  1, Batch 126 -Loss:  4439.8496 Validation Accuracy: 0.675781
Epoch  1, Batch 127 -Loss:  4440.4512 Validation Accuracy: 0.671875
Epoch  1, Batch 128 -Loss:  5487.0610 Validation Accuracy: 0.679688
Epoch  1, Batch 129 -Loss:  4755.6499 Validation Accuracy: 0.671875
Epoch  1, Batch 130 -Loss:  3871.6709 Validation Accuracy: 0.671875
Epoch  1, Batch 131 -Loss:  4443.2290 Validation Accuracy: 0.667969
Epoch  1, Batch 132 -Loss:  3154.1016 Validation Accuracy: 0.667969
Epoch  1, Batch 133 -Loss:  4627.5312 Validation Accuracy: 0.660156
Epoch  1, Batch 134 -Loss:  5474.5566 Validation Accuracy: 0.675781
Epoch  1, Batch 135 -Loss:  3861.4434 Validation Accuracy: 0.667969
Epoch  1, Batch 136 -Loss:  4878.5903 Validation

Epoch  1, Batch 243 -Loss:  2577.2993 Validation Accuracy: 0.750000
Epoch  1, Batch 244 -Loss:  3215.4541 Validation Accuracy: 0.753906
